## Price Data Scheduler

- Function Goal:
    * pull 48 hours of hourly price data once every hour 
- Tech:
    * python schedule library

In [53]:
# import dependencies
import pandas as pd
import numpy as np
import ccxt
import os
import requests
import json
import schedule
import time
import math
import datetime
from dotenv import load_dotenv


# import functions from other notebooks
from utils.utils import epoch_to_datetime

In [54]:
# load_dotenv()
# kucoin_apikey=os.getenv('KUCOIN_API_KEY')

In [55]:
url = "https://api.kucoin.com/api/v1/market/orderbook/level1?symbol=SUSD-USDT"
response = requests.get(url).json()
response
# response["data"]["price"]

{'code': '200000',
 'data': {'time': 1657768598124,
  'sequence': '634840',
  'price': '1.001',
  'size': '0.6943',
  'bestBid': '1.00101',
  'bestBidSize': '117',
  'bestAsk': '1.003389',
  'bestAskSize': '382.5901'}}

In [74]:
def get_kucoin_ohlcv(currency,sma_window,df=None):
    exchange = ccxt.kucoin()
    if df is None:
        df = pd.DataFrame()
    rows = df.shape[0]
    if rows > 1:
        df = df.iloc[1:,] # drop oldest row in df 
        print('Dropping oldest record ...')
    rows = df.shape[0]
    limit = sma_window-rows

    new_df = pd.DataFrame(exchange.fetchOHLCV(currency, timeframe = "1h", limit = limit , params={'price':'index'}))
    new_df.columns = ['epoch', 'open', 'high', 'low', 'close', 'volume']
    new_df['epoch'] = new_df['epoch']/1000  # from epoch in ms to epoch in seconds
    new_df['Time (UTC)'] = new_df['epoch'].apply(epoch_to_datetime)
    new_df.drop(columns=['epoch'], inplace = True)
    new_df.set_index('Time (UTC)', inplace = True)
    df = pd.concat([df, new_df])
    return df

In [75]:
cache_df = get_kucoin_ohlcv('SUSD/USDT', 42)
cache_df.shape

,0,1,2,3,4,5
0,1657620000000,1.002001,1.002001,1.002001,1.002001,8.6117
1,1657623600000,1.004690,1.004699,1.002001,1.004699,0.9503
2,1657627200000,1.004699,1.004699,1.004699,1.004699,0.0000
3,1657630800000,1.004699,1.004700,1.002010,1.004694,132.8030
4,1657634400000,1.004694,1.004694,1.004694,1.004694,0.0000
5,1657638000000,1.004694,1.004694,1.004694,1.004694,0.0000
6,1657641600000,1.004699,1.004699,1.004699,1.004699,0.1183
7,1657645200000,1.004698,1.004698,1.004698,1.004698,0.2218
8,1657648800000,1.002211,1.004697,1.002211,1.004697,1.3144
9,1657652400000,1.002202,1.004696,1.002200,1.002200,49.2106


(42, 5)

In [76]:
cache_df = get_kucoin_ohlcv('SUSD/USDT', 42, cache_df)
display(cache_df)
cache_df.shape

Dropping oldest record ...


,0,1,2,3,4,5
0,1657767600000,1.001,1.001,1.001,1.001,0.6943


,open,high,low,close,volume
Time (UTC),,,,,
2022-07-12 11:00:00+00:00,1.004690,1.004699,1.002001,1.004699,0.9503
2022-07-12 12:00:00+00:00,1.004699,1.004699,1.004699,1.004699,0.0000
2022-07-12 13:00:00+00:00,1.004699,1.004700,1.002010,1.004694,132.8030
2022-07-12 14:00:00+00:00,1.004694,1.004694,1.004694,1.004694,0.0000
2022-07-12 15:00:00+00:00,1.004694,1.004694,1.004694,1.004694,0.0000
2022-07-12 16:00:00+00:00,1.004699,1.004699,1.004699,1.004699,0.1183
2022-07-12 17:00:00+00:00,1.004698,1.004698,1.004698,1.004698,0.2218
2022-07-12 18:00:00+00:00,1.002211,1.004697,1.002211,1.004697,1.3144
2022-07-12 19:00:00+00:00,1.002202,1.004696,1.002200,1.002200,49.2106


(42, 5)

In [ ]:
#schedule.every().hour.at('00:00:01').do(get_kucoin_ohlcv)
schedule.every(10).seconds.do(get_kucoin_ohlcv)
while True:
    schedule.run_pending()
    time.sleep(1)